In [271]:
import scipy
from scipy import signal
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
import glob


# global variables
# directory with data
datadirectory = "./flute/philharmonia/"

# maximum of examples
max_ex = 5

# file names
ex_wav = glob.glob(datadirectory + '*.wav')[:max_ex]

# TODO: a human-readble version of ex_wav
# ex_names = 

# samplerate of the data
samplerate = 44100.
# bit depth of the data
bitdepth = np.int16
#parameter for tukey window
alpha_tukey = 0.05


# input: a .wav file 
# output: a vector of integers (one float per sample)
def import16monowav(wav_file) :
    samplerate, data = wavfile.read(wav_file)
    #return np.fromfile(open(wav_file),bitdepth)[24:]
    return data

# normalize to a float in [-1,1]
def normalize16bits(vector) :
    return [ sample/2**16. for sample in vector ]



def import_convert_transform(ex) :
    # import wav and convert to float
    ex_float = normalize16bits(import16monowav(ex))
    # apply tukey window
    ex_windowed = signal.tukey(len(ex_float),alpha=0.05) * ex_float  
    # transform to frequency domain (since this is a real signal the transformed vector has half of the length)
    ex_transformed_complex = np.fft.rfft(ex_windowed)
    # take absolute values
    ex_transformed = map(abs,ex_transformed_complex)
    # compute frequencies for the xlabel
    freq_label = np.fft.rfftfreq(len(ex_float),1/samplerate)
    
    return ex_float, ex_transformed, freq_label
    #freq_label = [ np.arange(len(ex))/(len(ex)*2/samplerate) for ex in ex_transformed ]

# more global variables
ex_floats, ex_transformed, freq_label = zip(*map(import_convert_transform, ex_wav))


In [272]:
# TODO: this should not be sooo hardcoded
peak_width = 40

# a quarter of tone
qt_tone = 2**(1/24.)


# range of harmonics of interest
relevant_range_min = 100
relevant_range_max = 10000
# how many harmonics
num_harmonics = 5 

def harmonics_energies(ex, frq_lbl) :
    # find peaks (this is the position in the array)
    peaks = signal.find_peaks_cwt(ex, np.arange(peak_width,peak_width+1))
    # filter peaks in range and 
    peaks_in_range = filter(lambda n : frq_lbl[n]>=relevant_range_min and frq_lbl[n]<= relevant_range_max, peaks)
    # calculate energy of peaks
    peaks_energy_tmp = [ np.average(ex[int(peak/qt_tone):int(peak*qt_tone)]) for peak in peaks_in_range ]
    # find tonic
    tonic_pos = np.argmax(peaks_energy_tmp)
    #take only num_harmonics peaks
    peaks_energy = peaks_energy_tmp[tonic_pos:tonic_pos+num_harmonics]
    #the frequency of the peaks
    peaks_frequency = [ frq_lbl[n] for n in peaks_in_range[tonic_pos:tonic_pos+num_harmonics] ]
    # calculate relative energy of peaks
    tonic_energy = peaks_energy[0]
    peaks_energy_relative = [ harmonic_energy/tonic_energy for harmonic_energy in peaks_energy ]
    return peaks_frequency, peaks_energy_relative


In [273]:
names_and_graphs = zip(ex_wav, ex_transformed, freq_label)

def legend_and_show() :
    plt.legend(loc='upper right')
    plt.show()

# plot (1/4 of the) spectrum 
def plotspec() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,NUM_EX)))
    for name, ex, freqs in names_and_graphs :
        c = next(color)
        howmuch = len(ex)/4
        plt.plot(freqs[:howmuch], ex[:howmuch], label=name,c=c)
    legend_and_show()
    
def plotspec_normalized() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,NUM_EX)))
    for name, ex, freqs in names_and_graphs :
        c = next(color)
        #normalize
        ex /= np.max(np.abs(ex),axis=0)
        howmuch = len(ex)/4
        plt.plot(freqs[:howmuch], ex[:howmuch], label=name,c=c)
    legend_and_show()

def plotspec_and_harmonics() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,NUM_EX)))
    for name, ex, freqs in names_and_graphs :
        c = next(color)
        #normalize
        ex /= np.max(np.abs(ex),axis=0)
        howmuch = len(ex)/4
        plt.plot(freqs[:howmuch], ex[:howmuch], label=name,c=c)
        x, y = harmonics_energies(ex, freqs)
        plt.scatter(x, y, label=name,c=c)
    legend_and_show()

def plotharmonics() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,NUM_EX)))
    for name, ex, freqs in names_and_graphs :
        c = next(color)
        x, y = harmonics_energies(ex, freqs)
        xx = range(len(y))
        plt.scatter(xx, y,label=name,c=c)
    legend_and_show()
    
def plotharmonics_exp() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,NUM_EX)))
    for name, ex, freqs in names_and_graphs :
        c = next(color)
        x, y = harmonics_energies(ex, freqs)
        yy = [ np.e**yi for yi in y ]
        xx = range(len(y))
        plt.scatter(xx, yy, label=name,c=c)
    legend_and_show()

def plotharmonics2() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,NUM_EX)))
    for name, ex, freqs in names_and_graphs :
        c = next(color)
        x, y = harmonics_energies(ex, freqs)
        plt.plot(y,label=name,c=c)
    legend_and_show()

In [ ]:
plotspec_and_harmonics()

In [258]:
#harmonics_energies(ex_transformed[0], freq_label[0])

peaks = signal.find_peaks_cwt(ex_transformed[1], np.arange(peak_width,peak_width+1))
peaks_in_range = filter(lambda n : freq_label[1][n]>=relevant_range_min and freq_label[1][n]<= relevant_range_max, peaks)
peaks_energy_tmp = [ np.average(ex[int(peak/qt_tone):int(peak*qt_tone)]) for peak in peaks_in_range ]
#peaks_energy_tmp = [ np.average(ex[peak]) for peak in peaks_in_range ]

peaks_frequency = [ freq_label[1][n] for n in peaks_in_range ]

zip(peaks_frequency, peaks_energy_tmp)


[(275.47487745098039, 0.032355278318714355),
 (291.23774509803923, 0.028618920976074685),
 (583.22610294117646, 0.020726326658631371),
 (857.19975490196077, 1.4261617265163853),
 (881.21936274509801, 1.1407908047135937),
 (1148.4375, 0.035750276554378679),
 (1164.2003676470588, 0.043614635142963487),
 (1171.7064950980391, 0.047456871404902186),
 (1459.9417892156862, 0.047332199122061706),
 (1732.4142156862745, 0.31325733945031847),
 (1747.4264705882354, 0.30467153600885466),
 (1759.436274509804, 0.28059806534560811),
 (2022.9013480392157, 0.02236610416983971),
 (2040.9160539215686, 0.031567730846823593),
 (2055.1776960784314, 0.039839139281214075),
 (2285.6158088235293, 0.013913110138156522),
 (2332.153799019608, 0.01444932537948616),
 (2474.7702205882351, 0.027130124962450625),
 (2626.3939950980393, 0.12921367144361898),
 (2657.9197303921569, 0.046583871220117423),
 (2675.9344362745096, 0.032945825323245241),
 (2686.4430147058824, 0.029302243485874013),
 (2703.7071078431372, 0.0252992

In [160]:
ex_wav

['./flute/philharmonia/flute_Gs5_15_forte_normal.wav',
 './flute/philharmonia/flute_D4_15_mezzo-piano_normal.wav',
 './flute/philharmonia/flute_D5_05_pianissimo_normal.wav',
 './flute/philharmonia/flute_F6_025_forte_normal.wav',
 './flute/philharmonia/flute_Ds7_025_piano_normal.wav',
 './flute/philharmonia/flute_Fs5_1_mezzo-piano_normal.wav',
 './flute/philharmonia/flute_A4_very-long_cresc-decresc_normal.wav']

In [43]:
#plt.plot(ex_windowed[0])
#plt.show()

#ex_windowed[0][len(ex_windowed[0])-200000:len(ex_windowed[0])]


#f, t, Sxx = signal.spectrogram(ex_floats[0], fs=samplerate, window=('tukey', 0.25))

#plt.pcolormesh(t, f, Sxx)
#plt.ylabel('Frequency [Hz]')
#plt.xlabel('Time [sec]')
#plt.show()

#plt.plot(spectro)
#plt.show()


In [ ]:
#smooth vector, but the peak finder already does this

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

average_factor = 30

ex_smoothened = [ smooth(ex, average_factor) for ex in ex_transformed ]
